In [ ]:
import os
import time
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [ ]:
video_dir = '/mnt/disks/datastorage/videos/'
weights_dir = '/mnt/disks/datastorage/weights/'

video_path = os.path.join(video_dir,"elp/demos","theft_b.mov")
flownet_model = 'FlowNet2CSS'
flownet_weights = os.path.join(weights_dir, 'FlowNet2-CSS.pth.tar')
spatial_weights = os.path.join(weights_dir, 'spatial_weights_custom.pth.tar')
motion_weights = os.path.join(weights_dir, 'motion_weights_custom.pth.tar')

In [3]:
!python pipeline.py --stream $video_path --model $flownet_model -ow $flownet_weights -sw $spatial_weights -mw $motion_weights

Parsing Arguments
  [0.221s] fp16: False
  [0.221s] fp16_scale: 1024.0
  [0.221s] image_size: [224, 224]
  [0.221s] inference_size: [-1, -1]
  [0.221s] loss: L1Loss
  [0.221s] model: FlowNet2CSS
  [0.221s] model_batchNorm: False
  [0.221s] model_div_flow: 20.0
  [0.221s] motion_weights: /mnt/disks/datastorage/weights/motion_weights_custom.pth.tar
  [0.221s] nb_classes: 4
  [0.221s] number_gpus: 1
  [0.221s] optical_weights: /mnt/disks/datastorage/weights/FlowNet2-CSS.pth.tar
  [0.221s] rgb_max: 255.0
  [0.221s] seed: 1
  [0.221s] spatial_weights: /mnt/disks/datastorage/weights/spatial_weights_custom.pth.tar
  [0.221s] stream: /mnt/disks/datastorage/videos/elp/demos/theft_b.mov
  [0.221s] Operation finished

Building FlowNet2CSS model
/home/mlp/LENS/flownet2-pytorch/models.py:460: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(m.bias)
/home/mlp/LENS/flownet2-pytorch/models.py:461: UserWarning: nn.init.xavier_uniform is now deprecated in favor

Processing frame 261
Processing frame 262
Processing frame 263
Processing frame 264
Processing frame 265
Processing frame 266
Processing frame 267
Processing frame 268
Processing frame 269
Processing frame 270
Processing frame 271
Processing frame 272
Processing frame 273
Processing frame 274
Processing frame 275
Processing frame 276
Processing frame 277
Processing frame 278
Processing frame 279
Processing frame 280
Processing frame 281
Processing frame 282
Processing frame 283
Processing frame 284
Processing frame 285
Processing frame 286
Processing frame 287
Processing frame 288
Processing frame 289
Processing frame 290
Processing frame 291
Processing frame 292
Processing frame 293
Processing frame 294
Processing frame 295
Processing frame 296
Processing frame 297
Processing frame 298
Processing frame 299
  [1.002m] Spatial network exiting
  [1.002m] Operation finished

  [1.778m] Temporal network exiting
  [1.778m] Operation finished

[array([[ 0.38005823,  0.11096735,  1.2449094 , 

In [ ]:
video_dir = os.path.dirname(os.path.abspath(video_path))
video_name = os.path.splitext(os.path.basename(video_path))[0]
pickle_path = os.path.join(video_dir, '{}_predictions.pkl'.format(video_name))

with open(pickle_path, 'rb') as pf:
    predictions = pickle.load(pf)
    predictions = np.array(predictions).squeeze()
    actions = np.argmax(predictions, axis=1)

In [ ]:
#for i in predictions:
#    print(i)

In [ ]:
action_dict = {
    0: 'Theft',
    1: 'Assault',
    2: 'Shooting',
    3: 'No Action',
}

cap = cv2.VideoCapture(video_path)

frame_time = 0
frame_idx = 0
display = None
i = 0
while True:
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break
    
    try:
        label = action_dict[actions[frame_idx]]
    except IndexError:
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.putText(
        frame,
        label,
        (10, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (255, 255, 255),
        thickness=3
    )
    
    if i < 50:
        frame_time += time.time() - start_time
        frame_idx += 1
        i+=1
        continue
        
        
    else:
        plt.axis('off')
        plt.imshow(frame)
        plt.show()
        clear_output(wait=True)

        frame_time += time.time() - start_time
        frame_idx += 1
cap.release()

frame_time /= frame_idx
print('Avg. frame processing time: {} s'.format(round(frame_time, 2)))